In [1]:
#imports
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [7]:
tf.__version__

'2.1.0'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# %cd drive/'My Drive'/nnfl-alexnet-data

/content/drive/My Drive/nnfl-alexnet-data


In [1]:
#helper functions

valData = np.load('./valDataComplete.npz')
inputVal = valData['arr_0']
outputVal = valData['arr_1']

trainData = np.load('./trainDataComplete.npz')
inputTrain = trainData['arr_0']
outputTrain = trainData['arr_1']




def pcaAug(original_image):
    #Takes in an image as a np array and returns the pca augmented image as a np array
    renorm_image = np.reshape(original_image,(original_image.shape[0]*original_image.shape[1],3))

    renorm_image = renorm_image.astype('float32')
    renorm_image -= np.mean(renorm_image, axis=0)
    renorm_image /= np.std(renorm_image, axis=0)

    cov = np.cov(renorm_image, rowvar=False)

    lambdas, p = np.linalg.eig(cov)
    alphas = np.random.normal(0, 0.1, 3)

    delta = np.dot(p, alphas*lambdas)

    delta = (delta).astype('int8')

    pca_color_image = np.maximum(np.minimum(original_image + delta, 255), 0).astype('uint8')
    return pca_color_image


def randomizeOrder(inArr, outArr, n):
    #Randomizes order of input Array and does the same for outputArray keeping relative positions same
    idx = np.array(range(n))
    np.random.shuffle(idx)
    inList = []
    outList = []
    for i in range(n):
        inList.append(inArr[idx[i]])
        outList.append(outArr[idx[i]])
    return inList, outList


inMean = 112.69858580973307
inStd = 70.93752549462194
#Constants calculated on the entire training dataset


def applyAug(img, outputOneHot):
    #Takes in an np array image and onHotEncoded output class and returns normalised and augmented images.
    #Normalises Image using the constants defined above
    #Takes five patches of the image, flips all of them, and applies PCA, creates 20 samples from 1 image
    #Output Image size is 56x56x3 from 64x64x3, the ratio is kept to be same as mentioned in the paper
    img = (img - inMean)/inStd
    arr = [img[0:56, 0:56, :], img[7:, 0:56, :], img[0:56, 7:, :], img[7:, 7:, :], img[3:59, 3:59, :]]
    arr = arr + [np.flip(arr[0], 1),np.flip(arr[1], 1), np.flip(arr[2], 1), np.flip(arr[3], 1), np.flip(arr[4], 1)]
    pcaArr = []
    for i in range(len(arr)):
        pcaArr.append(pcaAug(arr[i]))
    inputArr = arr + pcaArr
    outArr = []
    for i in range(len(inputArr)):
        outArr.append(outputOneHot)
    return inputArr, outArr  


def createTrainingSet():
    #Passes the entire data set to Augmentation function, structures the entire augmented data and 
    #randomizes the order using the randomizeOrder function
    inputArray = []
    outputArray = []
    for i in range(len(inputTrain)):
        tempIn, tempOut = applyAug(inputTrain[i], outputTrain[i])
        inputArray = inputArray + tempIn
        outputArray = outputArray + tempOut
    return randomizeOrder(inputArray, outputArray, len(inputArray))

def createValidationSet():
    #Passes the entire data set to Augmentation function, structures the entire augmented data and 
    #randomizes the order using the randomizeOrder function
    inputArray = []
    outputArray = []
    for i in range(len(inputVal)):
        tempIn, tempOut = applyAug(inputVal[i], outputVal[i])
        inputArray = inputArray + tempIn
        outputArray = outputArray + tempOut
    return randomizeOrder(inputArray, outputArray, len(inputArray))

augIn, augOut = createTrainingSet()

augInVal, augOutVal = createValidationSet()

def getTrainingSample():
    #wrapper to give data to the model in the desired API
    return [augIn, augOut]

def getValidationSample():
    #wrapper to give data to the model in the desired API
    return [augInVal, augOutVal]


NameError: name 'np' is not defined

In [ ]:
class LRN2D(tf.keras.layers.Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__(**kwargs)
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = K.shape(X)
        half_n = self.n // 2
        input_sqr = K.square(X)
        extra_channels = K.zeros((b, ch + 2 * half_n, r, c))
        input_sqr = K.concatenate([extra_channels[:, :half_n, :, :],
                                   input_sqr,
                                   extra_channels[:, half_n + ch:, :, :]],
                                  axis=1)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i + ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  "alpha": self.alpha,
                  "k": self.k,
                  "beta": self.beta,
                  "n": self.n}
        base_config = super(LRN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Model

In [3]:
def build(input_shape, num_classes):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(96, (11, 11), strides=(4, 4), padding='same', 
                                     activation='relu', use_bias = 1, 
                                     kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                     bias_initializer = 'zeros', input_shape=input_shape))
    model.add(LRN2D())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
    
    model.add(tf.keras.layers.Conv2D(256, (5, 5),
                                    activation='relu', use_bias = 1, 
                                    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                    bias_initializer = 'ones'))
    model.add(LRN2D())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', 
                                     activation='relu', use_bias = 1, 
                                     kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                     bias_initializer = 'zeros'))
    
    model.add(tf.keras.layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', 
                                     activation='relu', use_bias = 1, 
                                     kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                     bias_initializer = 'ones'))
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', 
                                     activation='relu', use_bias = 1, 
                                     kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                     bias_initializer = 'ones'))

    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(4096, activation='relu', use_bias = 1,
                                    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                    bias_initializer = 'ones'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(4096, activation='relu', use_bias = 1,
                                    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                    bias_initializer = 'ones'))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(num_classes, activation='softmax', use_bias = 1,
                                    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-2),
                                    bias_initializer = 'ones'))
    
    model.summary()
    tf.keras.utils.plot_model(model, to_file='architecture_imagenet.png')
              
    return model

In [4]:
def plot_graph(model, N):
    
    plt.figure(0)
    plt.plot(np.arange(0, N), model.history['loss'], label='Training loss')
    plt.plot(np.arange(0, N), model.history['val_loss'], label='Testing loss')
    plt.title('Training vs Testing Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.figure(1)
    plt.plot(np.arange(0, N), model.history['categorical_accuracy'], label='Top 1 Training Accuracy')
    plt.plot(np.arange(0, N), model.history['val_categorical_accuracy'], label='Top 1 Testing Accuracy')
    plt.title('Top 1 Training vs Testing Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.figure(2)
    plt.plot(np.arange(0, N), model.history['TopKCategoricalAccuracy'], label='Top 5 Training Accuracy')
    plt.plot(np.arange(0, N), model.history['val_TopKCategoricalAccuracy'], label='Top 5 Testing Accuracy')
    plt.title('Top 5 Training vs Testing Acuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

In [5]:
def model_callbacks(name):
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(name, monitor = 'val_categorical_accuracy',
    #                             mode = 'max', save_best_only = True, verbose = 1)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', factor = 0.1,    
                                  patience = 5, verbose = 1)
    return [reduce_lr]

In [6]:
# all the parameters
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.0005
total_epochs = 60
batch_size = 1000
input_shape = (56, 56, 3)
num_classes = 200

inp = np.array(getTrainingSample()[0])
out = np.array(getTrainingSample()[1])
inp_test = np.array(getValidationSample()[0])
out_test = np.array(getValidationSample()[1])

# Training TinyNet
model_tinynet = build(input_shape, num_classes)
model_tinynet.compile(optimizer = tfa.optimizers.weight_decay_optimizers.SGDW(
                        learning_rate=learning_rate, momentum=momentum, 
                        weight_decay=weight_decay, nesterov=True, name='SGDW'),
                        loss='categorical_crossentropy', 
                        metrics=['categorical_accuracy', 'TopKCategoricalAccuracy'])

history = model_tinynet.fit(inp, out, batch_size = batch_size, validation_data = (inp_test, out_test), 
                                      epochs = total_epochs, verbose=1,
                                      callbacks = model_callbacks('a'))

plot_graph(history, total_epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 256)         1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 384)         8

KeyboardInterrupt: 

In [0]:
x = getTrainingSample()

In [10]:
len(x[0])

400000

In [11]:
len(x[1])

400000

In [13]:
x[0][0].shape

(56, 56, 3)